In [1]:
from pyspark.sql import SQLContext
import json
from graphframes import *
import uuid
from graphframes.examples import Graphs
import requests
import time
import re
import wikipedia

In [2]:
def merge_graphframes(graphframes):
    print 'inside merge_graphframes: {}'.format(time.time())
    try:
        if(len(graphframes) > 0):
            v = graphframes[0].vertices
            e = graphframes[0].edges
        for i in range(len(graphframes)):
            if(i == 0):
                continue;
                
            for i in graphframes[i].vertices.groupBy("id").count():
                #get name field of current row , may not work see stackoverflow for sol
                name = graphframes[i].vertices[i].select("name")
                #check if it exists in v
                existing_node = v.filter("name = '{}'".format(name)).first()
                if existing_node != None:
                    new_vertex = sqlContext.createDataFrame([
                                    (name, name),
                                 ], ["id", "name"])
                    v = v.union(new_vertex)
                    
            for i in graphframes[i].edges.groupBy("src").count():
                #get src field of current row - may not work see stackoverflow for sol
                src = graphframes[i].vertices[i].select("src")
                src_id = v.filter("name = '{}'".format(src)).first()
                dst = graphframes[i].vertices[i].select("dst")
                dst_id = v.filter("name = '{}'".format(dst)).first()
                
                new_df = sqlContext.createDataFrame([
                            (text_name,
                             src_id,
                             dst_id,
                             rel[i],
                             temporal.get(sub[i] + rel[i] + obj[i], "None"), 
                             spatial.get(sub[i] + rel[i] + obj[i], "None"), 
                             purpose.get(sub[i] + rel[i] + obj[i], "None")) 
                         ], ["text_name", "src", "dst", "predicate", "temporal", "spatial", "purpose"])
                e = e.union(new_df)
                    
        print 'Safely exiting merge_graphframes {}'.format(time.time())
        return GraphFrame(v, e)
    except Exception as e:
        print e

In [4]:
def make_graphframe(graphene_output, text_name):
    print 'inside make_graphframe {} {}'.format(text_name, time.time())
    try:
        
        json_obj = json.loads(graphene_output.replace("'", '\"'))

        sentences = json_obj["sentences"]

        num_sentences = len(sentences)

        sub = []
        rel = []
        obj = []
        temporal = {}
        purpose = {}
        spatial = {}

        for i in range(num_sentences):
            sentence = sentences[i]
            extractionMap = sentence["extractionMap"]
            for key, val in extractionMap.items():
                sub.append(val["arg1"] if val["arg1"] else 'None')
                rel.append(val["relation"] if val["relation"] else 'None')
                obj.append(val["arg2"] if val["arg2"] else 'None')
                simpleContexts = val["simpleContexts"]
                num_contexts = len(simpleContexts)
                for j in range(num_contexts):
                    simpleContext = simpleContexts[j]
                    classification = simpleContext["classification"]
                    key = (val["arg1"] + val["relation"] + val["arg2"])
                    if (classification == 'TEMPORAL'):
                        temporal.update({val["arg1"] + val["relation"] + val["arg2"]:simpleContext["text"]})  #save in a Temporal hashmap like ["Obamaispresident"] : ["from 2017 to 2019"]
                    elif (classification == 'PURPOSE'):
                        purpose.update({val["arg1"] + val["relation"] + val["arg2"]:simpleContext["text"]})  #save in a Temporal hashmap like ["Obamaispresident"] : ["from 2017 to 2019"]
                    elif (classification == 'SPATIAL'):
                        spatial.update({val["arg1"] + val["relation"] + val["arg2"]:simpleContext["text"]})  #save in a Temporal hashmap like ["Obamaispresident"] : ["from 2017 to 2019"] 
        
        print len(temporal)
        print len(spatial)
        print len(purpose)
        
        for i in xrange(len(sub)):
            
            if i == 0:
                v = sqlContext.createDataFrame([
                        (sub[i], sub[i]),
                        (obj[i], obj[i]),
                    ], ["id", "name"])

                e = sqlContext.createDataFrame([
                        (text_name,
                         sub[i],
                         obj[i], 
                         rel[i], 
                         temporal.get(sub[i] + rel[i] + obj[i], "None"), 
                         spatial.get(sub[i] + rel[i] + obj[i], "None"), 
                         purpose.get(sub[i] + rel[i] + obj[i], "None"))
                    ], ["article_name", "src", "dst", "predicate", "temporal", "spatial", "purpose"])
            else:

                ########### Put sub[i] in vertex table if not present ################
                
                #See if already exist
                existing_sub = v.filter("name = '{}'".format(sub[i])).first()

                if existing_sub != None:
                    #sub[i] already exists
                    sub_id = None
                else:
                    sub_id = sub[i]

                if sub_id != None:
                    new_vertex = sqlContext.createDataFrame([
                                    (sub_id, sub[i]),
                                 ], ["id", "name"])
                    v = v.union(new_vertex)

                ########### Put obj[i] in vertex table if not present ################

                #See if already exist
                existing_obj = v.filter("name = '{}'".format(obj[i])).first()

                if existing_obj != None:
                    #obj[i] already exists
                    obj_id = None
                else:
                    obj_id = obj[i]


                if obj_id != None:
                    new_vertex = sqlContext.createDataFrame([
                                    (obj_id, obj[i]),
                                 ], ["id", "name"])
                    v = v.union(new_vertex)

                ############# Put entry in edge frame #########################

                sub_id = v.filter("name = '{}'".format(sub[i])).first().id
                obj_id = v.filter("name = '{}'".format(obj[i])).first().id
                
                new_df = sqlContext.createDataFrame([
                            (text_name,
                             sub_id,
                             obj_id,
                             rel[i],
                             temporal.get(sub[i] + rel[i] + obj[i], "None"), 
                             spatial.get(sub[i] + rel[i] + obj[i], "None"), 
                             purpose.get(sub[i] + rel[i] + obj[i], "None")) 
                         ], ["text_name", "src", "dst", "predicate", "temporal", "spatial", "purpose"])
                e = e.union(new_df)
        print 'safely exiting make_graphframe {} {}'.format(text_name, time.time())    
        return GraphFrame(v, e)
    except Exception as e:
        print e

In [5]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


def get_graphene_response(file):
    print '#################  Inside get_graphene_response {} {}'.format(file, time.time())
    try:
        
        
        cwd = os.getcwd()
        f=open( cwd + "/dataset/1000words/" + file + ".txt", "r")
        
        content = f.read()
        content = ''.join(e for e in content if (e.isalnum() or e == ' ' or e == '.'))
        payload = {"doCoreference": "false", "isolateSentences": "false"}
        payload['text'] = content
        headers = {"Content-Type": "application/json", "Accept": "application/json"}
        
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=3)
        session.mount('http://', adapter)
        url = 'http://34.96.81.129/relationExtraction/text' #container-native
        
        response = session.post(url, data=json.dumps(payload), headers=headers, timeout=2000)
        while(response.status_code!=200):
            response = session.post(url, data=json.dumps(payload), headers=headers, timeout=2000)
            
        print '###########  Safely now exiting get_graphene_response {} {}'.format(file, time.time())
        print response
        
        return response.text
    except Exception as e:
        print  e


In [3]:
def get_knowledge_graph(article_dict):
    graphframes = []
    for key,value in article_dict.iteritems():
        graphene_response = get_graphene_response(value)
        graphframes.append(make_graphframe(graphene_response, key))
        
    return merge_graphframes(graphframes)

In [6]:
def get_filtered_result(graphframe, conditions = []):
    filtered = graphframe.edges
    for x in conditions:
        print x
        filtered = filtered.filter(x)
    return filtered

In [7]:
def get_selected_columns(filtered_result, columns = []):
    selected = filtered_result
    selected = selected.select(columns)
    return selected

In [9]:
def get_filtered_result(graphframe, conditions = []):
    filtered = graphframe.edges
    vertices = graphframe.vertices
    for x in conditions:
        str = x.split()
        col = str[0]
        if (col == 'src'):
            x2 = x.replace('src', 'name')
            vertices.filter(x2)
            joinExp = vertices["id"] == filtered["src"]
            filtered= filtered.join(vertices, joinExp, 'inner')
        elif (col == 'dst'):
            x2 = x.replace('dst', 'name')
            vertices.filter(x2)
            joinExp = vertices["id"] == filtered["dst"]
            filtered= filtered.join(vertices, joinExp, 'inner')
        else:
            filtered = filtered.filter(x)
    return filtered

In [10]:
def get_selected_columns(filtered_result, columns = []):
    cols = []
    for i in range(len(columns)):
        if(columns[i].strip() == ('s')):
            cols.append("src")
        if(columns[i].strip() == ('p')):
            cols.append("predicate")
        if(columns[i].strip() == ('o')):
            cols.append("dst")
        if(columns[i].strip() == ('temporal')):
            cols.append("temporal")
        if(columns[i].strip() == ('spatial')):
            cols.append("spatial")
        if(columns[i].strip() == ('purpose')):
            cols.append("purpose")
        if(columns[i].strip() == ('*')):
            cols.append("*")
    selected = filtered_result
    selected = selected.select(cols)
    return selected
    

In [12]:

query_string1 = """*
                 src ~Facebook
                 predicate ~founded"""

print time.time()

conditions = []
split_query = query_string1.splitlines()
if '*' in split_query:
    select_cols = '*'
else:    
    select_cols = split_query[0].strip().split(',')

    
graph = graphframes[0]
    
for i in range(len(split_query)):
    if i == 0:
        continue
    c = split_query[i].strip().split(' ')
    
    col_name = c[0]
    if(c[1] == "="):
        conditions += (col_name + '=' + "'" + c[2] + "'")
    elif c[1].find('~')>-1:
        val = c[1].split('~')
        if len(c)>2:
            val[1] = val[1]+' '+ c[2]+' '+c[3]
        conditions.append(col_name + ' like ' + "'%" + val[1] + "%'")
        
filtered_result = get_filtered_result(graph, conditions)
final_output = get_selected_columns( filtered_result, select_cols)

print time.time()

1567023024.19
*


NameError: name 'graphframes' is not defined